In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu"

In [2]:
data1=pd.read_csv("./data/goodtraining.csv")
data2=pd.read_csv("./data/badtraining.csv")

data_train = pd.concat([data1,data2])
data_train.head(200)


,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,645.819778,228.747883,-495.751920,833.777542,466.400785,-179.431511,453.908920,460.894146,-190.957317,good
1,645.382538,231.751356,-479.768572,833.793488,460.541124,-157.067810,454.630623,459.233665,-191.068661,good
2,637.317886,198.295734,-426.740828,833.420715,440.161572,-157.098591,453.729401,428.858485,-178.784831,good
3,651.948700,228.353620,-363.429108,846.754837,433.019085,-107.618058,496.846695,431.808615,-130.351560,good
4,658.113556,241.229854,-425.602069,853.639832,431.371436,-108.308308,509.732590,433.925028,-124.991133,good
...,...,...,...,...,...,...,...,...,...,...
52,697.492981,456.650805,-1259.881897,937.801208,560.484567,-733.029699,440.948715,559.970655,-712.199106,bad
53,698.278809,554.905014,-1393.053789,965.860138,592.686954,-910.816984,428.239441,585.115786,-860.147696,bad
54,711.930389,516.182199,-1226.822147,952.692108,584.226494,-714.547477,431.635818,578.647885,-715.498095,bad
55,704.784088,480.823388,-1308.877058,950.093689,593.617144,-674.223318,395.330734,615.484443,-705.777769,bad


In [3]:

data3=pd.read_csv("./data/goodvalidation.csv")
data4=pd.read_csv("./data/badvalidation.csv")

data_test = pd.concat([data3,data4])
data_test.head(200)

,Nose X,Nose Y,Nose Z,Left Shoulder X,Left Shoulder Y,Left Shoulder Z,Right Shoulder X,Right Shoulder Y,Right Shoulder Z,Label
0,629.449768,287.503946,-433.363738,804.088211,473.737378,-177.699909,483.490562,482.585363,-174.759039,good
1,703.323135,287.549565,-363.598237,879.282303,479.812174,-168.614194,528.779068,483.545852,-141.752150,good
2,656.710587,202.545190,-441.115794,867.903748,429.677567,-206.222520,464.457474,429.084349,-195.042815,good
3,736.965332,278.175781,-723.139172,911.553497,470.187292,-381.313992,519.369125,462.966356,-415.945730,good
4,664.094315,245.109444,-588.598452,900.551300,461.928535,-284.289458,472.788200,459.991164,-292.915313,good
5,698.764801,268.351686,-703.565011,929.103317,479.501209,-411.444726,472.503510,483.020782,-381.039248,good
6,691.072845,267.272794,-1056.093407,916.090240,536.329579,-590.278587,419.694862,558.924079,-564.323301,good
7,694.683380,282.088287,-550.334444,906.751709,529.089289,-267.421453,423.971329,538.596325,-269.863486,good
0,715.415955,584.799371,-1032.254019,932.152252,583.881583,-685.010691,519.674759,569.154925,-617.172647,bad
1,713.711777,620.993657,-1183.147202,966.189499,688.436537,-787.063293,488.173752,689.148030,-748.276234,bad


In [4]:
data = pd.concat([data_train,data_test])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import torch.utils

encoder = LabelEncoder()
data["Label"] = encoder.fit_transform(data["Label"])
X=data.drop(columns="Label").to_numpy(dtype=np.float32)
y=data["Label"].to_numpy(dtype=np.float32)
scaler=MinMaxScaler()
X=scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

train_dataset= torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))

test_dataset= torch.utils.data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))


train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=4,num_workers=3)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=3)

In [6]:

class slouch_detection(nn.Module):
    def __init__(self):
        super(slouch_detection,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(9,36),
            nn.ReLU(),
            nn.Linear(36,15),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(15,12),
            nn.ReLU(),
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,3),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.layers(x)


In [7]:
model = slouch_detection().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Assuming you're using Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # You can adjust the learning rate as needed


In [15]:
total=0
for params in model.parameters():
    total+=params.numel()
total

1210

In [12]:
def train_model(model, train_loader, optimizer, criterion, epochs,device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            optimizer.zero_grad()
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            #print(outputs.size(),labels.unsqueeze(dim=1).size())
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            
            # Calculate accuracy
            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            print()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        tqdm.write(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Testing loop
def test_model(model, test_loader, criterion,device,best_acc):
    model.eval()
    with torch.inference_mode():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in tqdm(test_loader, desc='Testing'):
            labels=labels.unsqueeze(dim=1).to(device)
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            predicted = torch.round(outputs)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        test_loss = running_loss / len(test_loader.dataset)
        test_acc = correct / total
        if(best_acc<test_acc):
            torch.save(model.state_dict(),"slouch_detector.pt")
            best_acc=test_acc
        tqdm.write(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [13]:
train_model(model, train_dataloader, optimizer, criterion,device=device, epochs=100)  # Adjust epochs as needed
best_acc=0
test_model(model, test_dataloader, criterion,device,best_acc)

Epoch 1/100: 100%|██████████| 19/19 [00:00<00:00, 81.05it/s]





















Epoch 1/100, Loss: 0.5314, Accuracy: 0.7534


Epoch 2/100: 100%|██████████| 19/19 [00:00<00:00, 106.75it/s]





















Epoch 2/100, Loss: 0.4976, Accuracy: 0.7671


Epoch 3/100:  53%|█████▎    | 10/19 [00:00<00:00, 97.73it/s]

...




















Epoch 99/100, Loss: 0.5026, Accuracy: 0.6986


Epoch 100/100: 100%|██████████| 19/19 [00:00<00:00, 76.36it/s] 





















Epoch 100/100, Loss: 0.4880, Accuracy: 0.7397


Testing: 100%|██████████| 37/37 [00:00<00:00, 157.68it/s]

Test Loss: 0.5900, Test Accuracy: 0.6757
